In [1]:
# from utils.compression import compress_directories, decompress_directories

# for base_path, batch_type in data_partition.items():
#     decompress_directories(
#         source_path=f'downloaded_data/{base_path}',
#         destination_path=f'data/{base_path}',
#         checksum_file_name="checksums.txt",
#         delete_compressed_files=True,
#         num_workers=26
#     )

In [ ]:
# del session.logger
# del session

from pickle import NONE
from train.training_session_v0 import TrainingSessionV0
from config import TrainingConfigV0
import multiprocessing
from config import SimpleConvConfig
from models.simpleconv import SimpleConv
import torch

data_partition = {
    "gwilliams2023": {
        "testing_subjects": [19, 20, 21],
        "testing_tasks": [0],
    },
    # "armeini2022": {
    #     "testing_subjects": [],
    #     "testing_tasks": [8, 9],
    # },
}

model_config = SimpleConvConfig(
    # Str to list of possible conditions
    mel_normalization=False,
    conditions={
        "study": [],
        "subject": [],
    },
    # Channels
    in_channels=208,
    out_channels=128,
    hidden_dim=384,
    dropout=0.2,
    initial_batch_norm=True,
    # Sensor layout settings
    layout_dim=2,
    layout_proj=True,
    layout_scaling="minmax",
    # Merger with spatial attn
    merger=False,
    merger_emb_type=None,
    merger_emb_dim=0,
    merger_channels=0,
    merger_dropout=False,
    merger_conditional=None,
    # Inital
    initial_linear=384,
    initial_depth=1,
    # Conditional layers
    conditional_layers=False,
    conditional_layers_dim=None,  # input or hidden_dim
    # Conv layer overall structure
    depth=6,
    kernel_size=3,
    growth=1.0,
    dilation_growth=2,
    dilation_period=5,
    glu=1,
    conv_dropout=0.2,
    dropout_input=0.2,
    batch_norm=True,
    half=False,
    # Quantizer
    quantizer=False,
    num_codebooks=0,
    codebook_size=0,
    quantizer_commitment=0,
    quantizer_temp_init=0,
    quantizer_temp_min=0,
    quantizer_temp_decay=0,
    # Transformers Encoders
    transformer_input=None,
    transformer_encoder_emb=None,
    transformer_encoder_layers=0,
    transformer_encoder_heads=0,
    # Transformer Decoders
    transformer_decoder_emb=None,
    transformer_decoder_layers=0,
    transformer_decoder_heads=0,
    transformer_decoder_dim=0,
)

config = TrainingConfigV0(
    brain_encoder_config=model_config,
    data_partition=data_partition,
    # Pre-processing parameters
    # Brain
    new_freq=100,
    frequency_bands={"all": (0.5, 40)},
    max_random_shift=1.0,
    window_size=4,
    window_stride=1,
    brain_clipping=None,
    baseline_window=0.0,
    notch_filter=True,
    scaling="standard",
    delay=0.15,
    # Hyperparameters
    learning_rate=3e-4,
    weight_decay=1e-4,
    epochs=50,
    batch_size=256,
    use_clip_loss=True,
    use_mse_loss=True,
    alpha=0.6,
    random_test_size=10,
    seed=42,
)

session = TrainingSessionV0(
    config=config,
    studies={study: "audio" for study in data_partition.keys()},
    data_path="data",
    save_path="saves/phase1/objectives/CLIP_MSE",
    clear_cache=False,
    cache_name="cache/1",
)

try:
    session.train(
        device="cuda",
        buffer_size=30,
        num_workers=(multiprocessing.cpu_count() - 2) // 2,
        max_cache_size=400,
        current_epoch=0,
    )
except KeyboardInterrupt as e:
    print("Exited")

Loading Gwilliams2023 with batch type audio
Data partitioned on studies ['gwilliams2023'].
Train: 135, Unseen Task: 12, Unseen Subject: 45, Unseen Both: 4.

GPU is not NVIDIA V100, A100, or H100. Speedup numbers may be lower than expected.
SimpleConv initialized with 8448160 parameters, cond: ['study', 'subject']
Merger False, merger channels 0
ConvBlocks: 6, hidden_dim: 384, params 7973376


Training Epoch 1: 100%|██████████| 135/135 [05:06<00:00,  2.27s/it]


Epoch 1 completed. Loss: 3.3592, Clip Loss: 5.0375, MSE Loss: 0.8417
Accuracy: 0.0499, Top 5: 0.1583, Top 10: 0.2434
Test unseen_subject completed. Accuracy: 0.0705, Top 5: 0.2010, Top 10: 0.3080
Test unseen_task completed. Accuracy: 0.0576, Top 5: 0.1725, Top 10: 0.2664
Test unseen_both completed. Accuracy: 0.0712, Top 5: 0.1737, Top 10: 0.2587
Testing completed in 0.46m.
Epoch 1 completed in 5.56m. 0.04m per recording.


Training Epoch 2: 100%|██████████| 135/135 [05:15<00:00,  2.34s/it]


Epoch 2 completed. Loss: 2.7826, Clip Loss: 4.3002, MSE Loss: 0.5063
Accuracy: 0.0900, Top 5: 0.2550, Top 10: 0.3697
Test unseen_subject completed. Accuracy: 0.1105, Top 5: 0.2735, Top 10: 0.3825
Test unseen_task completed. Accuracy: 0.1018, Top 5: 0.2851, Top 10: 0.4047
Test unseen_both completed. Accuracy: 0.0963, Top 5: 0.2675, Top 10: 0.4113
Testing completed in 0.46m.
Epoch 2 completed in 5.72m. 0.04m per recording.


Training Epoch 3: 100%|██████████| 135/135 [05:04<00:00,  2.26s/it]


Epoch 3 completed. Loss: 2.6340, Clip Loss: 4.0594, MSE Loss: 0.4959
Accuracy: 0.1246, Top 5: 0.3179, Top 10: 0.4400
Test unseen_subject completed. Accuracy: 0.0690, Top 5: 0.2445, Top 10: 0.3415
Test unseen_task completed. Accuracy: 0.1009, Top 5: 0.2669, Top 10: 0.3892
Test unseen_both completed. Accuracy: 0.0675, Top 5: 0.2175, Top 10: 0.3312
Testing completed in 0.41m.
Epoch 3 completed in 5.49m. 0.04m per recording.


Training Epoch 4: 100%|██████████| 135/135 [04:32<00:00,  2.02s/it]


Epoch 4 completed. Loss: 2.5610, Clip Loss: 3.9382, MSE Loss: 0.4951
Accuracy: 0.1466, Top 5: 0.3553, Top 10: 0.4781
Test unseen_subject completed. Accuracy: 0.1240, Top 5: 0.3255, Top 10: 0.4415
Test unseen_task completed. Accuracy: 0.1398, Top 5: 0.3666, Top 10: 0.4925
Test unseen_both completed. Accuracy: 0.1225, Top 5: 0.3275, Top 10: 0.4375
Testing completed in 0.36m.
Epoch 4 completed in 4.90m. 0.04m per recording.


Training Epoch 5:  93%|█████████▎| 126/135 [04:30<00:14,  1.59s/it]

## CLIP MSE

In [ ]:
# del session.logger
# del session

from pickle import NONE
from train.training_session_v0 import TrainingSessionV0
from config import TrainingConfigV0
import multiprocessing
from config import SimpleConvConfig
from models.simpleconv import SimpleConv
import torch

data_partition = {
    "gwilliams2023": {
        "testing_subjects": [19, 20, 21],
        "testing_tasks": [0],
    },
    # "armeini2022": {
    #     "testing_subjects": [],
    #     "testing_tasks": [8, 9],
    # },
}

model_config = SimpleConvConfig(
    # Str to list of possible conditions
    mel_normalization=False,
    conditions={
        "study": [],
        "subject": [],
    },
    # Channels
    in_channels=208,
    out_channels=128,
    hidden_dim=384,
    dropout=0.2,
    initial_batch_norm=True,
    # Sensor layout settings
    layout_dim=2,
    layout_proj=True,
    layout_scaling="minmax",
    # Merger with spatial attn
    merger=False,
    merger_emb_type=None,
    merger_emb_dim=0,
    merger_channels=0,
    merger_dropout=False,
    merger_conditional=None,
    # Inital
    initial_linear=384,
    initial_depth=1,
    # Conditional layers
    conditional_layers=False,
    conditional_layers_dim=None,  # input or hidden_dim
    # Conv layer overall structure
    depth=6,
    kernel_size=3,
    growth=1.0,
    dilation_growth=2,
    dilation_period=5,
    glu=1,
    conv_dropout=0.2,
    dropout_input=0.2,
    batch_norm=True,
    half=False,
    # Quantizer
    quantizer=False,
    num_codebooks=0,
    codebook_size=0,
    quantizer_commitment=0,
    quantizer_temp_init=0,
    quantizer_temp_min=0,
    quantizer_temp_decay=0,
    # Transformers Encoders
    transformer_input=None,
    transformer_encoder_emb=None,
    transformer_encoder_layers=0,
    transformer_encoder_heads=0,
    # Transformer Decoders
    transformer_decoder_emb=None,
    transformer_decoder_layers=0,
    transformer_decoder_heads=0,
    transformer_decoder_dim=0,
)

config = TrainingConfigV0(
    brain_encoder_config=model_config,
    data_partition=data_partition,
    # Pre-processing parameters
    # Brain
    new_freq=100,
    frequency_bands={"all": (0.5, 40)},
    max_random_shift=1.0,
    window_size=4,
    window_stride=1,
    brain_clipping=None,
    baseline_window=0.0,
    notch_filter=True,
    scaling="standard",
    delay=0.15,
    # Hyperparameters
    learning_rate=3e-4,
    weight_decay=1e-4,
    epochs=50,
    batch_size=256,
    use_clip_loss=True,
    use_mse_loss=True,
    alpha=0.6,
    random_test_size=10,
    seed=42,
)

config.notch_filter = False

session = TrainingSessionV0(
    config=config,
    studies={study: "audio" for study in data_partition.keys()},
    data_path="data",
    save_path="saves/phase1/ablation/notch_filter/False",
    clear_cache=True,
    cache_name="cache/1",
)

try:
    session.train(
        device="cuda",
        buffer_size=30,
        num_workers=(multiprocessing.cpu_count() - 2) // 2,
        max_cache_size=400,
        current_epoch=0,
    )
except KeyboardInterrupt as e:
    print("Exited")

## No notch